In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"C:\Users\hp\Downloads\TRADES_CopyTr_90D_ROI.csv")

In [3]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [4]:
df.shape

(150, 2)

In [5]:
df["Trade_History"].iloc[1]

"[{'time': 1718980078000, 'symbol': 'NEARUSDT', 'side': 'SELL', 'price': 5.344, 'fee': -0.138944, 'feeAsset': 'USDT', 'quantity': 277.888, 'quantityAsset': 'USDT', 'realizedProfit': 7.02899999, 'realizedProfitAsset': 'USDT', 'baseAsset': 'NEAR', 'qty': 52.0, 'positionSide': 'BOTH', 'activeBuy': False}, {'time': 1718976178000, 'symbol': 'NEARUSDT', 'side': 'BUY', 'price': 5.183, 'fee': -0.046647, 'feeAsset': 'USDT', 'quantity': 93.294, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'NEAR', 'qty': 18.0, 'positionSide': 'BOTH', 'activeBuy': True}, {'time': 1718974114000, 'symbol': 'NEARUSDT', 'side': 'BUY', 'price': 5.212, 'fee': -0.00521199, 'feeAsset': 'USDT', 'quantity': 10.424, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'NEAR', 'qty': 2.0, 'positionSide': 'BOTH', 'activeBuy': True}, {'time': 1718974114000, 'symbol': 'NEARUSDT', 'side': 'BUY', 'price': 5.212, 'fee': -0.033878, 'feeAsset': 'USD

In [6]:
df['Trade_History'].head()

0    [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1    [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2    [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3    [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4    [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
Name: Trade_History, dtype: object

In [7]:
print(type(df['Trade_History'][0]))

<class 'str'>


In [8]:
import ast

# Converting stringified lists into actual lists
df['Trade_History'] = df['Trade_History'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [9]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [10]:
print(type(df['Trade_History'][0]))

<class 'list'>


In [11]:
# Exploding the Trade_History column
df_exploded = df.explode('Trade_History', ignore_index=True)
print(df_exploded.head())

              Port_IDs                                      Trade_History
0  3925368433214965504  {'time': 1718899656000, 'symbol': 'SOLUSDT', '...
1  3925368433214965504  {'time': 1718899618000, 'symbol': 'DOGEUSDT', ...
2  3925368433214965504  {'time': 1718899618000, 'symbol': 'DOGEUSDT', ...
3  3925368433214965504  {'time': 1718899616000, 'symbol': 'DOGEUSDT', ...
4  3925368433214965504  {'time': 1718899616000, 'symbol': 'DOGEUSDT', ...


In [12]:
# Flattening the exploded dictionary into individual columns
trade_details = pd.json_normalize(df_exploded['Trade_History'])

# Combining the normalized data with other columns
df_final = df_exploded.drop(columns=['Trade_History']).join(trade_details)

In [13]:
df_final.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False
2,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False
3,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True
4,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True


In [14]:
# renaming the column 
df_final.rename(columns={"quantity": "Investment"}, inplace=True)

In [15]:
df_final.head(2)

,Port_IDs,time,symbol,side,price,fee,feeAsset,Investment,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False


In [16]:
# df_final.rename(columns={"realizedProfit": "PnL"}, inplace=True)

In [17]:
# Investment per trade
df_final['ROI'] = (df_final['realizedProfit'] / df_final['Investment']) * 100  # ROI per trade

# 1 for winning trade, -1 for loosing 
df_final['winning_trade'] = df_final['realizedProfit'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


In [18]:
df_final.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,Investment,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,ROI,winning_trade
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,0.0,0
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,0.0,0
2,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False,0.0,0
3,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True,0.0,0
4,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True,0.0,0


In [19]:
df2 = df_final.copy()

In [20]:
# Step 1: Group by portfolio_id for the realizedProfit feature
df2['portfolio_value'] = df2.groupby('Port_IDs')['realizedProfit'].cumsum()  # Cumulative sum of profits per portfolio

# Step 2: Calculating the running max for each portfolio
df2['running_max'] = df2.groupby('Port_IDs')['portfolio_value'].cummax()

# Step 3: Calculating the drawdown for each portfolio_id
df2['drawdown'] = (df2['portfolio_value'] - df2['running_max']) / df2['running_max']


In [21]:
df2[:15]

,Port_IDs,time,symbol,side,price,fee,feeAsset,Investment,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,ROI,winning_trade,portfolio_value,running_max,drawdown
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,0.0,0,0.0,0.0,NaN
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,0.0,0,0.0,0.0,NaN
2,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False,0.0,0,0.0,0.0,NaN
3,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True,0.0,0,0.0,0.0,NaN
4,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True,0.0,0,0.0,0.0,NaN
5,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.125718,USDT,251.43648,USDT,0.0,USDT,DOGE,2064.0,LONG,True,0.0,0,0.0,0.0,NaN
6,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.021806,USDT,43.61156,USDT,0.0,USDT,DOGE,358.0,LONG,True,0.0,0,0.0,0.0,NaN
7,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.58500,-0.159102,USDT,795.51000,USDT,0.0,USDT,SOL,6.0,LONG,False,0.0,0,0.0,0.0,NaN
8,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.58500,-0.596633,USDT,1193.26500,USDT,0.0,USDT,SOL,9.0,LONG,True,0.0,0,0.0,0.0,NaN
9,3925368433214965504,1.718890e+12,FILUSDT,BUY,4.45400,-0.348525,USDT,697.05100,USDT,0.0,USDT,FIL,156.5,LONG,True,0.0,0,0.0,0.0,NaN


In [22]:
df2.tail()

,Port_IDs,time,symbol,side,price,fee,feeAsset,Investment,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,ROI,winning_trade,portfolio_value,running_max,drawdown
211273,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035968,-0.025070,USDT,50.139392,USDT,0.0,USDT,JASMY,1394.0,BOTH,False,0.0,0,243.668899,243.668899,0.0
211274,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035968,-0.008255,USDT,16.509312,USDT,0.0,USDT,JASMY,459.0,BOTH,False,0.0,0,243.668899,243.668899,0.0
211275,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035969,-0.031545,USDT,63.089626,USDT,0.0,USDT,JASMY,1754.0,BOTH,False,0.0,0,243.668899,243.668899,0.0
211276,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035969,-0.031905,USDT,63.809006,USDT,0.0,USDT,JASMY,1774.0,BOTH,False,0.0,0,243.668899,243.668899,0.0
211277,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035972,-0.005018,USDT,10.036188,USDT,0.0,USDT,JASMY,279.0,BOTH,False,0.0,0,243.668899,243.668899,0.0


In [23]:
df2.iloc[1200]

Port_IDs               3925368433214965504
time                       1715094060000.0
symbol                             BCHUSDT
side                                  SELL
price                               487.42
fee                              -0.138184
feeAsset                              USDT
Investment                       276.36714
quantityAsset                         USDT
realizedProfit                         0.0
realizedProfitAsset                   USDT
baseAsset                              BCH
qty                                  0.567
positionSide                         SHORT
activeBuy                            False
ROI                                    0.0
winning_trade                            0
portfolio_value                5318.748719
running_max                    5318.748719
drawdown                               0.0
Name: 1200, dtype: object

In [24]:
df2.shape

(211278, 20)

In [25]:
df2.isna().sum()

Port_IDs                  0
time                      1
symbol                    1
side                      1
price                     1
fee                       1
feeAsset                  1
Investment                1
quantityAsset             1
realizedProfit            1
realizedProfitAsset       1
baseAsset                 1
qty                       1
positionSide              1
activeBuy                 1
ROI                       1
winning_trade             0
portfolio_value           1
running_max               1
drawdown               1713
dtype: int64

In [26]:
## substituing nan with 0 at drawdown because for some portfolios nan is huddle to find the minimum
df2["drawdown"].fillna(0, inplace=True)

In [27]:
type(df2["drawdown"].iloc[24])

numpy.float64

In [28]:
df2["drawdown"].iloc[24]

0.0

In [29]:
df2.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,Investment,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,ROI,winning_trade,portfolio_value,running_max,drawdown
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,0.0,0,0.0,0.0,0.0
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,0.0,0,0.0,0.0,0.0
2,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False,0.0,0,0.0,0.0,0.0
3,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True,0.0,0,0.0,0.0,0.0
4,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True,0.0,0,0.0,0.0,0.0


In [30]:
# Step 4: Calculating MDD for each portfolio_id
mdd_per_portfolio = df2.groupby('Port_IDs')['drawdown'].min()

In [31]:
print("Maximum Drawdown (MDD) for each portfolio_id:")
mdd_per_portfolio

Maximum Drawdown (MDD) for each portfolio_id:


Port_IDs
3672754654734989568    -0.460781
3733192481840423936    -0.760355
3768170840939476993     0.000000
3784403294629753856    -0.177261
3786761687746711808    -2.318212
                         ...    
4039279455324236544     0.000000
4040382575336130560     0.000000
4040843843196854529     0.000000
4041804592937345281   -46.067384
4041860229502600193     0.000000
Name: drawdown, Length: 150, dtype: float64

In [32]:
df_final.isna().sum()

Port_IDs               0
time                   1
symbol                 1
side                   1
price                  1
fee                    1
feeAsset               1
Investment             1
quantityAsset          1
realizedProfit         1
realizedProfitAsset    1
baseAsset              1
qty                    1
positionSide           1
activeBuy              1
ROI                    1
winning_trade          0
dtype: int64

In [33]:
df_final.dropna(inplace=True, axis=0)

In [34]:
df_final.head(2)

,Port_IDs,time,symbol,side,price,fee,feeAsset,Investment,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,ROI,winning_trade
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,0.0,0
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,0.0,0


In [35]:
total_positions = df_final.groupby('Port_IDs').size()

win_rate = df_final.groupby('Port_IDs')['winning_trade'].mean()

df_final['daily_return'] = df_final['realizedProfit']  # Assuming realizedProfit is daily for simplicity

# Calculating mean and standard deviation of daily returns per portfolio_id
mean_returns = df_final.groupby('Port_IDs')['daily_return'].mean()
std_returns = df_final.groupby('Port_IDs')['daily_return'].std()

# Sharpe Ratio = (mean returns - risk-free rate) / standard deviation
# Assuming risk-free rate = 0
sharpe_ratio = mean_returns / std_returns


In [36]:
sharpe_ratio

Port_IDs
3672754654734989568    0.185274
3733192481840423936    0.060265
3768170840939476993    0.424277
3784403294629753856    0.106585
3786761687746711808    0.215150
                         ...   
4039279455324236544    0.471684
4040382575336130560         NaN
4040843843196854529    0.341668
4041804592937345281   -0.077929
4041860229502600193    0.457193
Name: daily_return, Length: 149, dtype: float64

In [37]:
total_realized_profit = df_final.groupby('Port_IDs')['realizedProfit'].sum()
total_investment = df_final.groupby('Port_IDs')['Investment'].sum()

# ROI = (Total Realized Profit / Total Investment) * 100
roi = (total_realized_profit / total_investment) * 100

# PnL is the sum of realizedProfit for each portfolio_id
pnl = total_realized_profit

# Count how many trades have a positive realizedProfit for each portfolio_id
winning_positions = df_final[df_final['winning_trade'] == 1].groupby('Port_IDs').size()

In [45]:
# Output all metrics
metrics = pd.DataFrame({
    "ROI": roi,
    "PnL": pnl,
    'Total Positions': total_positions,
    'Win Rate': win_rate * 100,
    "Win Positions": winning_positions,
    'Sharpe Ratio': sharpe_ratio,
    "MDD": mdd_per_portfolio
})

In [46]:
metrics

,ROI,PnL,Total Positions,Win Rate,Win Positions,Sharpe Ratio,MDD
Port_IDs,,,,,,,
3672754654734989568,0.476385,566.597660,474.0,24.683544,210.0,0.185274,-0.460781
3733192481840423936,0.251099,2923.977200,689.0,79.245283,553.0,0.060265,-0.760355
3768170840939476993,8.779089,243.668899,14.0,42.857143,6.0,0.424277,0.000000
3784403294629753856,0.339819,2521.814305,6050.0,23.537190,1829.0,0.106585,-0.177261
3786761687746711808,0.332072,205.021400,82.0,40.243902,37.0,0.215150,-2.318212
...,...,...,...,...,...,...,...
4039279455324236544,1.022101,1038.807419,327.0,55.351682,181.0,0.471684,0.000000
4040382575336130560,0.000000,0.000000,76.0,0.000000,NaN,NaN,0.000000
4040843843196854529,0.985523,2151.704060,59.0,32.203390,19.0,0.341668,0.000000


In [47]:
metrics.isna().sum()

ROI                1
PnL                1
Total Positions    1
Win Rate           1
Win Positions      2
Sharpe Ratio       2
MDD                0
dtype: int64

In [48]:
metrics[["Win Positions", "Sharpe Ratio"]].replace(0, np.nan, inplace=True)

In [49]:
metrics.isna().sum()

ROI                1
PnL                1
Total Positions    1
Win Rate           1
Win Positions      2
Sharpe Ratio       2
MDD                0
dtype: int64

In [50]:
metrics.dropna(inplace=True, axis=0)

In [51]:
metrics.to_csv("portfolio_metrics.csv", index=True)